In [1]:
import csv
task_to_scenes = {}
with open(r"C:\Users\cgokmen\Downloads\b1k-tasks.csv", "r") as f:
    for row in csv.DictReader(f):
        scene_list_str = row["Matched Planned Scenes"].replace(" ", "").replace("\n", "").strip()
        task_scenes = scene_list_str.split(",")
        task_to_scenes[row["Task Name"]] = {x for x in task_scenes if x}

In [2]:
len(task_to_scenes)

1034

In [3]:
scenes = {scene for task_scenes in task_to_scenes.values() for scene in task_scenes}
print(scenes)

{'Rs_garden', 'hotel_suite_large', 'office_vendor_machine', 'restaurant_cafeteria', 'Benevolence_2_int', 'school_gym', 'Benevolence_0_int', 'office_bike', 'restaurant_hotel', 'office_cubicles_right', 'Beechwood_0_int', 'grocery_store_cafe', 'restaurant_brunch', 'house_double_floor_upper', 'restaurant_diner', 'house_double_floor_lower', 'house_single_floor', 'hotel_gym_spa', 'Merom_0_garden', 'school_geography', 'Benevolence_1_int', 'Wainscott_0_garden', 'hotel_suite_small', 'Pomaria_0_garden', 'Pomaria_2_int', 'hall_train_station', 'Pomaria_0_int', 'Pomaria_1_int', 'office_cubicles_left', 'Beechwood_0_garden', 'hall_conference_large', 'school_chemistry', 'Beechwood_1_int', 'hall_arch_wood', 'grocery_store_half_stocked', 'gates_bedroom', 'Wainscott_1_int', 'Merom_1_int', 'office_large', 'restaurant_asian', 'Merom_0_int', 'Ihlen_0_int', 'hall_glass_ceiling', 'school_computer_lab_and_infirmary', 'Rs_int', 'school_biology', 'Wainscott_0_int', 'Ihlen_1_int', 'restaurant_urban', 'grocery_sto

In [4]:
# Starting pick
ig_scenes = {x for x in scenes if x.endswith("_int")}
starting_set = ig_scenes | {
    "house_single_floor",
    "house_double_floor_lower",
    "house_double_floor_upper",
}

In [5]:
# What activities do these scenes cover?
def cover(scene_set):
    return [task for task, task_scenes in task_to_scenes.items() if task_scenes & scene_set]

len(cover(starting_set))

868

In [6]:
# Greedy algorithm step
def next_pick(scene_set):
    candidates = scenes - scene_set
    current_cover = len(cover(scene_set))
    advantages = {x: len(cover(scene_set | {x})) - current_cover for x in candidates}
    # print(sorted(advantages.items(), key=lambda x: x[1]))
    return max(advantages.items(), key=lambda x: x[1])
next_pick(starting_set)

('grocery_store_cafe', 41)

In [7]:
# Run the greedy algorithm
max_cover = len(cover(scenes))
greedy_selection = set(starting_set)
while len(cover(greedy_selection)) < max_cover:
    pick = next_pick(greedy_selection)
    print("Adding", pick)
    greedy_selection.add(pick[0])
print("Final set:", sorted(greedy_selection))
print(len(greedy_selection))

Adding ('grocery_store_cafe', 41)
Adding ('Rs_garden', 21)
Adding ('restaurant_brunch', 17)
Adding ('office_large', 7)
Adding ('hotel_gym_spa', 4)
Adding ('school_geography', 4)
Adding ('Pomaria_0_garden', 3)
Adding ('hotel_suite_large', 1)
Adding ('grocery_store_half_stocked', 1)
Adding ('school_computer_lab_and_infirmary', 1)
Final set: ['Beechwood_0_int', 'Beechwood_1_int', 'Benevolence_0_int', 'Benevolence_1_int', 'Benevolence_2_int', 'Ihlen_0_int', 'Ihlen_1_int', 'Merom_0_int', 'Merom_1_int', 'Pomaria_0_garden', 'Pomaria_0_int', 'Pomaria_1_int', 'Pomaria_2_int', 'Rs_garden', 'Rs_int', 'Wainscott_0_int', 'Wainscott_1_int', 'grocery_store_cafe', 'grocery_store_half_stocked', 'hotel_gym_spa', 'hotel_suite_large', 'house_double_floor_lower', 'house_double_floor_upper', 'house_single_floor', 'office_large', 'restaurant_brunch', 'school_computer_lab_and_infirmary', 'school_geography']
28


In [8]:
import itertools, tqdm

print("Max possible cover:", max_cover)

# Run a non-greedy version
def combinatorial_search():
    max_extra_scenes_needed = len(greedy_selection) - len(starting_set)
    print("We will process up to", max_extra_scenes_needed, "extra scenes")
    extra_scene_candidates = scenes - starting_set
    print("We have", len(extra_scene_candidates), "candidates")
    
    def combinatorial_search_n(extra_to_use):
        best_cover = 0
        best_cover_set = set()
        
        print("Processing", extra_to_use, "element combinations")
        for combination_extra in tqdm.tqdm(itertools.combinations(extra_scene_candidates, extra_to_use)):
            combination = starting_set | set(combination_extra)
            combination_cover = len(cover(combination))

            if combination_cover >= best_cover:
                best_cover = combination_cover
                best_cover_set = combination
            
            if combination_cover == max_cover:
                break
                
        return best_cover_set, best_cover
    
    for extra_to_use in range(1, max_extra_scenes_needed + 1):
        combination, combination_cover = combinatorial_search_n(extra_to_use)
        print(f"Best total with {extra_to_use} elements: {combination_cover}. Scenes: {sorted(combination - starting_set)}")
        
        if combination_cover == max_cover:
            print("Final set.")
            return
            
combinatorial_search()

Max possible cover: 968
We will process up to 10 extra scenes
We have 33 candidates
Processing 1 element combinations


33it [00:00, 2869.95it/s]


Best total with 1 elements: 909. Scenes: ['grocery_store_cafe']
Processing 2 element combinations


528it [00:00, 2659.99it/s]


Best total with 2 elements: 930. Scenes: ['Rs_garden', 'grocery_store_cafe']
Processing 3 element combinations


5456it [00:02, 2671.24it/s]


Best total with 3 elements: 947. Scenes: ['Rs_garden', 'grocery_store_cafe', 'restaurant_brunch']
Processing 4 element combinations


3324it [00:01, 2528.72it/s]


KeyboardInterrupt: 

In [ ]:
# Pomaria-advantage
print(len(cover(starting_set | {"Pomaria_0_garden"})) - len(cover(starting_set)))

In [9]:
final_set = starting_set | {'Pomaria_0_garden', 'grocery_store_cafe', 'office_large', 'restaurant_brunch'}
len(cover(final_set))

952

In [ ]:
print(final_set - ig_scenes)